In [2]:
from google.colab import drive
import sys

drive.mount('/content/gdrive/')
path = '/content/gdrive/MyDrive/Caminho Pro Projeto no seu Drive/'
sys.path.append(path)
# path='./'

## Registra os nomes de métricas e sensores a receberem informações

In [3]:
from api import send_api
import requests

with open('token') as file: token = file.read()
headers = {
    'Authorization': f'Bearer {token}',
    'Content-Type': 'application/json'
}

# Estas métricas
metrics = [
    "label_parado_nox",
    "anomaly_consumption",
    "anomaly_liter_supply",
    "anomaly_km_driven",
    "anomaly_nox",
    "anomaly_o2",
]

# Serão enviadas para os sensores com estes prefixos
sensor_prefixes = [
    'Deteccao_Movimento_',
    'Anomalias_Combustivel_',
    'Anomalias_Combustivel_',
    'Anomalias_Combustivel_',
    'Anomalias_Combustivel_',
    'Anomalias_Combustivel_',
]

Envia a informação e registra um log

In [5]:
# Com um header contendo autorização, requisita os nomes de sensores válidos
def get_registered_sensors(auth, ambiente='DEV'):
    url = "https://iot-backend.sapienz.alis.solutions/registry/automation/sensor/supermercadomundial"
    
    response = requests.request("GET", url, headers=auth, data={})
    print(response.text)
    response = response.json()
    sensor_names = []

    for i in response['items']:
        id = i['compoundId']['identifier']
        sensor_names.append(id)
    return sensor_names

# Com um header contendo autorização, requisita a relação numero:placa
# Isso é importante para elaborar sensores hipotéticos
def get_number_plate(auth, ambiente="DEV"):
    url = "https://iot-backend.sapienz.alis.solutions/registry/automation/monitored_object/supermercadomundial" 

    response = requests.request("GET", url, headers=auth, data={})
    consulta = response.json()
    number_plate = {}
    print(consulta)
    for item in consulta["items"]:
        truck_data = item["customData"]
        frota = truck_data["fleet_number"]
        placa = truck_data["plate"]
        if frota != '': number_plate[frota] = placa
    return number_plate

from api import sensor_register

# Current Sensors
current_sensors = get_registered_sensors(auth=headers, ambiente='PROD')

# Every vehicle number and plate relationship
number_plate = get_number_plate(auth=headers, ambiente='PROD')

print(f'Sensores atuais, {current_sensors}')
# Create a match for a sensor
for metric, prefix in zip(metrics, sensor_prefixes):
    df, metric = send_api.retrieve_metric(path, metric)

    # Make up the sensor names
    candidates = []
    for vehicle_number, data in df.groupby('vehicle_number'):
        plate = number_plate[vehicle_number]
        candidates.append((prefix+plate, 'WM_'+plate))

    # Checks whether or not they are registered
    # If not, lists their name and monitored objects
    missing_sensors = []
    monitored_objects = []

    print(f'Sensores Faltando para a métrica {metric}')
    for possible_sensor, monitored_object in candidates:
        if not (possible_sensor in current_sensors):
            missing_sensors.append(possible_sensor)
            monitored_objects.append(monitored_object)
    print(missing_sensors)


{"pageIndex":0,"currentSize":85,"totalSize":85,"items":[{"customData":{"disp":"WM_FDZ2964"},"compoundId":{"domainId":"whitemartins","identifier":"ecu_WM_FDZ2964"},"providerId":"alisv2","metricSourceIdentifier":"truck_ecu_WM_FDZ2964"},{"customData":{"disp":"WM_FDZ2965"},"compoundId":{"domainId":"whitemartins","identifier":"ecu_WM_FDZ2965"},"providerId":"alisv2","metricSourceIdentifier":"truck_ecu_WM_FDZ2965"},{"customData":{"disp":"WM_FEI3446"},"compoundId":{"domainId":"whitemartins","identifier":"ecu_WM_FEI3446"},"providerId":"alisv2","metricSourceIdentifier":"truck_ecu_WM_FEI3446"},{"customData":{"disp":"WM_RIT2C79"},"compoundId":{"domainId":"whitemartins","identifier":"ecu_WM_RIT2C79"},"providerId":"alisv2","metricSourceIdentifier":"truck_ecu_WM_RIT2C79"},{"customData":{"disp":"WM_LTS8E38"},"compoundId":{"domainId":"whitemartins","identifier":"ecu_WM_LTS8E38"},"providerId":"alisv2","metricSourceIdentifier":"truck_ecu_WM_LTS8E38"},{"customData":{"disp":"WM_NZU2560"},"compoundId":{"dom